# GAN háló felépítése, tanítása és egyebek

Ezen notebook a GAN hálókról kíván egy kis összefoglalást nyújtani. 

## Áttekintés

A Generative Adverserial Network (GAN), vagy magyarul: Generatív Kontradiktórius Hálózat egy olyan generatív modell, amely nem a megszokott statisztikai alapokon optimalizál, mint például az autoencoder vagy pixelRNN modellek, hanem játékelméleti megközelítést alkalmaz.
A tanulás során két neurális hálózat versenyzik egymással: egy generátor, amelynek az a szerepe, hogy a tanítómintákhoz hasonló adatot generáljon a bemeneti zajból és egy diszkriminátor, amely a legegyszerűbb esetben egy bináris osztályozó, amely a generátor által generált adatot vizsgálja és eldönti, hogy az valódi vagy hamis. A tanítás során optimális esetben minden egyes epoch-kal egyre valósághűbb adatot állít elő a generátor és ezzel egyidejűleg is a diszkriminátor is fejlődni fog, tehát egyre pontosabban tudja megállapítani a valós és hamis adatokat és arra ösztönzi a generátort, hogy ő is minél pontosabban állítsa elő a hamis adatokat és így tovább...

## Tanítási folyamat

### Adathalmaz
A tanításhoz szükségünk van egy megfelelő adathalmazra.
Legegyszerűbb esetben elegendő lehet képek rendezetlen halmaza is és a modellre bízhatjuk, hogy ismerje fel az egyes osztályok jellegzetességeit. Megfelelő regularizációs technikákkal igazán változatosan ki lehet tölteni a látens teret. Viszont annak feltérképezése már nem evidens feladat. Hiszen képeket a látens tér egyes pontjainak kijelölésével és a ponthoz tartozó generátor kimenettel tudunk generálni. Vagyis a generátor segítségével mintavételezhetjük a látens teret. Viszont arról nem rendelkezünk előzetes információkkal, hogy a modell hogyan töltötte ki ezt a teret.

Ha az adathalmaz rendelkezik osztályokkal is, úgy az osztály-címkéket is felhasználhatjuk a GAN hálózatunk tanításához. Így egy plusz bemenet segítségével könnyebben tudunk majd megfelelő képeket előállítani.

Egyes adathalmazokhoz igen részletes annotációkat is mellékelnek. A képeken megfigyelhető objektumokat határoló dobozokkal, vagy pixel szinten jelölik. Így igen részletes információkat kínálnak a kép tartalmáról.
Egy igazán hasznos annotációk lehetnek a természetes nyelvű leíró mondatok is, amelyekből általában többet is mellékelnek egy-egy képhez.

### Tanítás

Ha a legegyszerűbb esetet vizsgáljuk és csak a képek rendezetlen halmazára tanítjuk a modellt, mindenféle kiegészítő információ és annotáció nélkül, akkor a tanítás a következőképpen zajlik:
A tanítómintát érdemes úgy összeállítani, hogy feldarabolható legyen, ezzel is megkönnyítve a párhuzamos feldolgozást. Az ilyen modellek tanításához általában igen sok képre van szükség és a teljes tanítóminta beadása a modell tanítását ellehetetlenítheti. Ha egy lépésben láthatná a diszkriminátor az összes képet, akkor túlságosan is megnőne a teljesítménye és a generátornak esélye sem lenne felzárkózni. Ehelyett a tanítóminta darabjain célszerű tanítani a GAN hálózatot, úgynevezett minibatch-okon. Így az egyes darabok feldolgozása után történne meg a generátor és a diszkriminátor frissítése, lehetőséget adva a generátornak, hogy megfelelő ütemben tanuljon.
Viszont ha minél változatosabb képeket generáló GAN hálózatot kívánunk létrehozni, úgy egyes cikkek szerint(melyek azok?) nagy batch mérettel kell tanítani a hálót.
Egy Epoch alatt a tanítóhalmaz összes batch-én való tanítást értjük.

A továbbiakban a következő jelöléseket használom: legyen $D$ a _Diszkriminátor_, $G$ pedig a _Generátor_.

#### Hibafüggvények

A _Generátor_ és _Diszkriminátor_ hibájának számolása a bináris kereszt-entrópián alapszik.

A bináris kereszt-entrópia hibafüggvény a következőképpen írható fel:
$$L(\hat y, y) = y . \log \hat y + (1-y). \log (1 - \hat y)$$
Ahol $\hat y$ a predikció, $y$ pedig a valós címke

##### Diszkriminátor hibafüggvénye
A _Generátor_ egy z bemeneti zajvektor alapján előállít egy generált adatot $G(z)$

A Diszkriminátor egy bináris osztályozó, amelynek feladata, hogy az $x$ és $G(z)$ bemeneteit osztályozza. $D(x)$ esetén 1, $D(G(z))$ esetén pedig 0 címkét várunk.

Így a kereszt-entrópia hibafüggvény $D(x)$ esetében a következő:
$$L(D(x), 1) = 1.\log D(x) + (1 - 1).\log(1 - D(x))$$
$$L(D(x), 1) = \log D(x)$$

A _Diszkriminátornak_ a $\log(D(x))$-et kell maximalizálnia.

$D(G(z))$ esetében a kereszt-entrópia hibafüggvény a következő:
$$L(D(G(z)), 0) = 0.\log D(G(z)) + (1 - 0).\log(1 - D(G(z)))$$
$$L(D(G(z)), 0) = \log(1- D(G(z)))$$

Vagyis a $\log(1- D(G(z)))$-t kell maximalizálnia

Egyetlen mintára a hibafüggvény a következőképpen néz ki:

$$\max V(D) = \log D(x) + \log(1 - D(G(z))$$


Batch-ra nézve:

$$\max V(D) = \mathbb{E}_{x \sim P(x))} \left[\log D(x) \right] + \mathbb{E}_{z \sim P(z))} \left[\log(1 - D(G(z))) \right]$$

Ahol a $P(x)$ a valószínűségi eloszlása a tanítóhalmaznak, $P(z)$ a valószínűségi eloszlása a $z$ zajvektornak. (látens tér).

##### Generátor hibafüggvénye
A _Generátor_ feladata az, hogy megtévessze a _Diszkriminátort_ azáltal, hogy a tanítóhalmazhoz hasonló adatokat generáljon.
Vagyis a _Generátor_ érdeke az, hogy a $D(G(z))$ 1-es címkét kapjon a _Diszkriminátortól_ 0 helyett.

Tehát a bináris keresztentrópia egy mintára:
$$L(D(G(z)), 0) = \log(1 - D(G(z))$$

Tehát a _Diszkriminátor_ minimalizálni kívánja a $D(G(z))$-t, míg a _Generátor_ maximalizálni szándékozik azt.

A _Generátor_ a tanítás során sosem fog valódi adatot látni, de a teljesség kedvééert a hibafüggvénye a következőképpen írható fel (Csak a második kifejezést minimalizálja valójában):

$$\min V(G) = \mathbb{E}_{x \sim P(x))} \left[\log D(x) \right] + \mathbb{E}_{z \sim P(z))} \left[\log(1 - D(G(z))) \right]$$


Vagyis a GAN hálózat tanítása során $D$ és $G$ egy minimax játékot játszanak a $V(G, D)$ értékfüggvénnyel.

$$\min_{G}\max_{D}V(D, G) =  \mathbb{E}_{x \sim P(x))} \left[\log D(x) \right] + \mathbb{E}_{z \sim P(z))} \left[\log(1 - D(G(z))) \right]$$


#### Optimalizáló módszer
A Generátorban is és a Diszkriminátoban is az Adam optimalizálót használtam. (Kezdeti paraméterek, learning rate, beta1-2?)

#### Tanítási lépés
%% GAN 2014 cikkből

A GAN hálózat egy tanítási lépése a következő lépésekből áll:

Legyen $m$ a minibatch elemszáma $m \in \mathbb{N}$

1. Hozzunk létre $m$ darab zajmintát $(z_1, \ldots, z_m)$ standard-normális eloszlásból $P_g(z)$.
2. A tanítóhalmazból emeljük ki a soronkövetkező $m$ darab tanítómintát (képet), és ezt jelöljük $(x_1, \ldots, x_m)$-el $P_{\text{data}}(x)$
3. Frissítsük a _Diszkriminátort_ a sztochasztikus gradiens emelkedésével (?? tükörfordítás)
$$ \nabla \theta_d \frac{1}{m} \sum_{i=1}^{m} \left[\log D(x_i) + \log(1 - D(G(z_i))) \right]$$
4. Frissítsük a _Generátort_ a sztochasztikus gradiens lejtésével (?? tükörfordítás)
$$ \nabla \theta_d \frac{1}{m} \sum_{i=1}^{m} \log(1 - D(G(z_i)))$$


Természetesen nem egyszerre tanítjuk a GAN hálózat részeit. Az eredeti cikkben is javaslatot tesznek arra, hogy a _Diszkriminátort_ esetleg több lépésben is lehetne tanítani, majd a _Generátort_ egyetlen lépésben frissíteni.
Különböző tanítási stratégiákban ez is egy szabad paraméter lehet. Számomra megfelelő volt az 1:1-es tanítási lépés alkalmazása is... (Esetleg lehetne mérni valahogy, hogy van-e számottevő különbség...)
A tanítás hossza természetesen függ az adathalmaz méretétől, a batch mérettől, a modellben található paraméterektől és az optimalizáló függvénytől.


## GAN teljesítményének mérese, technikák
- Inception Score (salimans2016improved, barratt2018note)
- Fréchet Inception Distance (heusel2017gans)

## Data augmentation - tanítás kevés adattal
- karras2020training
- noguchi2019image

## Regularizációs módszerek
(Még csak a Batch Normalization-t próbáltam az alapokon kívül)

A GAN tanítása során nehézségekbe ütközhetünk. Legrosszabb esetben nem is kezd el konvergálni a generátor kimenete a tanítóminta képeihez, viszont abban az esetben gyanakodhatunk, hogy a modellünk nem lett helyesen felépítve. Amennyiben mégis elkezd fejlődni a modell és nem csak zaj jelenik meg a kimeneten az egyes tanítólépések után, de bizonyos számú epoch után mode collapse lép fel, úgy fontolóra vehetjük a következő regularizációs technikákat.

### Megfelelő inicializációs stratégia

### Ativációs függvények és  paramétereiknek a helyes megválasztása

### Batch normalization

### Spectral Normalization

### Experience replay

### Mini-batch discrimination